In [37]:
import requests
from datetime import datetime, date, timedelta
import csv, os, re, time
from selenium import webdriver
from bs4 import BeautifulSoup
import csv

In [32]:
url = "https://www.eppo.go.th/epposite/index.php/th/petroleum/price/oil-price?orders%5BpublishUp%5D=publishUp&issearch=1"
html_page = requests.get(url)

print(html_page)
print(type(html_page))


<Response [200]>
<class 'requests.models.Response'>


In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time

url = "http://www.eppo.go.th/epposite/templates/eppo_v15_mixed/eppo_oil/eppo_oil_gen_new.php"

driver = webdriver.Chrome()
driver.get(url)
time.sleep(2)  # รอโหลด

soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

oil_div = soup.find("div", class_="div_oil_price")
print(oil_div is not None)  


True


## วันปัจจุบัน

In [ ]:
url = "http://www.eppo.go.th/epposite/templates/eppo_v15_mixed/eppo_oil/eppo_oil_gen_new.php"

driver = webdriver.Chrome()
driver.get(url)
time.sleep(2)

soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

oil_div = soup.find("div", class_="div_oil_price")

# ===== ดึงวันที่จาก header =====
date_header = oil_div.find("div", style=re.compile("font-size: 2em"))
raw_date = date_header.get_text(strip=True).replace("Oil price", "").strip()
date = datetime.strptime(raw_date, "%d %b %Y").date()

# ===== หา Gasohol 95 =====
g95_img = oil_div.find("img", {"src": re.compile("oil_name2.png")})
g95_row = g95_img.find_parent("div", class_=re.compile("oil_price_colum_name_(odd|even)"))
g95_prices = [col.text.strip() for col in g95_row.find_all("div", class_="oil_price_colum")]

# ===== หา Effective date =====
eff_label = None
for div in oil_div.find_all("div", class_="oil_price_colum_name"):
    if "Effective date" in div.get_text():
        eff_label = div
        break

eff_row = eff_label.find_parent("div", class_=re.compile("oil_price_colum_name_(odd|even)"))
eff_dates = [col.text.strip() for col in eff_row.find_all("div", class_="oil_price_colum")]

# ===== Mapping 3 ปั๊ม =====
stations = ["PTT", "Bangchak", "Shell"]
indices = [0, 1, 2]

result = {
    "date": date,
    "data": {
        station: {
            "price_gasohol_95": g95_prices[i],
            "effective_date": eff_dates[i]
        }
        for station, i in zip(stations, indices)
    }
}

print(result)

# วันนี้แบบไม่ใช้ API

{'date': datetime.date(2025, 9, 7), 'data': {'PTT': {'price_gasohol_95': '32.95', 'effective_date': '22 Aug 05:00'}, 'Bangchak': {'price_gasohol_95': '32.95', 'effective_date': '22 Aug 05:00'}, 'Shell': {'price_gasohol_95': '33.45', 'effective_date': '22 Aug 05:00'}}}


In [50]:
rows = []
for station, i in zip(stations, indices):
    rows.append([
        date,          # วันที่ scrape
        station,             # ชื่อปั๊ม
        g95_prices[i],       # ราคาน้ำมัน Gasohol 95
        eff_dates[i]         # Effective date
    ])

# ===== บันทึก CSV =====
with open("oil_prices.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["date", "station", "price_gasohol_95", "effective_date"])
    writer.writerows(rows)

print("✅ บันทึกข้อมูลลงไฟล์ oil_prices.csv เรียบร้อยแล้ว")

✅ บันทึกข้อมูลลงไฟล์ oil_prices.csv เรียบร้อยแล้ว


## ย้อนหลัง 3 วัน

In [51]:
def scrape_price(date_str):
    """Scrape ราคาน้ำมันจาก eppo_oil_gen_new.php ตามวันที่ที่กำหนด (dd/mm/yyyy)"""
    url = f"http://www.eppo.go.th/epposite/templates/eppo_v15_mixed/eppo_oil/eppo_oil_gen_new.php?TbxToDate={date_str}"
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    oil_div = soup.find("div", class_="div_oil_price")

    # ===== หา Gasohol 95 =====
    g95_img = oil_div.find("img", {"src": re.compile("oil_name2.png")})
    g95_row = g95_img.find_parent("div", class_=re.compile("oil_price_colum_name_(odd|even)"))
    g95_prices = [col.text.strip() for col in g95_row.find_all("div", class_="oil_price_colum")]

    # ===== หา Effective date =====
    eff_label = None
    for div in oil_div.find_all("div", class_="oil_price_colum_name"):
        if "Effective date" in div.get_text():
            eff_label = div
            break
    eff_row = eff_label.find_parent("div", class_=re.compile("oil_price_colum_name_(odd|even)"))
    eff_dates = [col.text.strip() for col in eff_row.find_all("div", class_="oil_price_colum")]

    stations = ["PTT", "Bangchak", "Shell"]
    indices = [0, 1, 2]

    # แปลง date เป็น ISO
    date = datetime.strptime(date_str, "%d/%m/%Y").strftime("%Y-%m-%d")
    year = date.split("-")[0]

    # แปลง effective_date
    eff_dates_iso = []
    for eff in eff_dates:
        if eff == "-":
            eff_dates_iso.append("")
        else:
            eff_dt = datetime.strptime(f"{eff} {year}", "%d %b %H:%M %Y")
            eff_dates_iso.append(eff_dt.strftime("%Y-%m-%d %H:%M"))

    result = []
    for station, i in zip(stations, indices):
        result.append({
            "date": date,
            "station": station,
            "price_gasohol_95": g95_prices[i],
            "effective_date": eff_dates_iso[i]
        })
    return result


# ====== scrap ย้อนหลัง 3 วันจาก "วันนี้" ======
today = date.today()   # ✅ ใช้วันที่ปัจจุบันแบบ real time
all_data = []

for i in range(1,4):
    day = today - timedelta(days=i)
    date_str = day.strftime("%d/%m/%Y")
    all_data.extend(scrape_price(date_str))


In [ ]:
# ====== บันทึก CSV ======
file_name = "oil_prices.csv"
file_exists = os.path.isfile(file_name)

with open(file_name, "a", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["date", "station", "price_gasohol_95", "effective_date"])
    if not file_exists:
        writer.writeheader()
    writer.writerows(all_data)

print(f"✅ เก็บข้อมูล {len(all_data)} แถว จาก {today} ย้อนหลัง 3 วัน บันทึกลง oil_prices.csv แล้ว")

✅ เก็บข้อมูล 9 แถว จาก 2025-09-07 ย้อนหลัง 3 วัน บันทึกลง oil_prices.csv แล้ว
